In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [2]:
!pip install bert-extractive-summarizer
!pip install rouge
from rouge import Rouge
import sys
import string
import nltk
from tqdm import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from summarizer import Summarizer, TransformerSummarizer

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires tensorboard<2.7,>=2.6.0, but you have tensorboard 2.10.1 which is incompatible.
tensorflow 2.6.4 requires typing-extensions<3.11,>=3.7, but you have typing-extensions 4.1.1 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires te

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [3]:
val_df = pd.read_csv('../input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')
test_df = pd.read_csv('../input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')

In [4]:
!mkdir test

In [5]:
!mkdir val

In [6]:
print(val_df.duplicated(subset=['article', 'highlights']).sum())
val_df = val_df.drop_duplicates(subset=['article','highlights'])
val_df.reset_index()
print(val_df.shape)

0
(13368, 3)


In [7]:
print(test_df.duplicated(subset=['article', 'highlights']).sum())
test_df = test_df.drop_duplicates(subset=['article','highlights'])
test_df.reset_index()
print(test_df.shape)

2
(11488, 3)


In [8]:
val_df['article_len'] = val_df.apply(lambda r: len(r['article'].split()), axis=1)
val_df['highlights_len'] = val_df.apply(lambda r: len(r['highlights'].split()), axis=1)
val_df = val_df.drop(columns=['id'])

In [9]:
test_df['article_len'] = test_df.apply(lambda r: len(r['article'].split()), axis=1)
test_df['highlights_len'] = test_df.apply(lambda r: len(r['highlights'].split()), axis=1)
test_df = test_df.drop(columns=['id'])

In [10]:
k_list = [1, 2, 3, 4, 5]

In [11]:
for k in tqdm(k_list):
    art_ext_list = []
    for i in tqdm(val_df.index, position=0):
        news = val_df.loc[i]
        art = news['article']
        high = news['highlights']
        art = sent_tokenize(art)
        orig_art = art

        art_form = []
        stop = stopwords.words('english')
        punc = string.punctuation

        for sent in art:
            sent_form = []
            words = word_tokenize(sent)
            for word in words:
                if word not in stop and word not in punc:
                    sent_form.append(word.lower())
            art_form.append(sent_form)

        art_form_1 = []
        art_form_len = []
        lemma = WordNetLemmatizer()

        for sent in art_form:
            words = []
            art_form_len.append(len(sent))
            for word in sent:
                lemma_word = lemma.lemmatize(word)
                words.append(lemma_word)
            art_form_1.append(words)

        art_form = art_form_1

        art_form_1 = []
        for sen in art_form:
            s = " ".join(sen)
            art_form_1.append(s)

        art_form = art_form_1

        vect = TfidfVectorizer()
        tf_idf_mat = vect.fit_transform(art_form)
        score = np.sum(tf_idf_mat, axis=1)
        score = np.ravel(score)
        n = max(int(len(art_form_len)/k),1)
        idx = np.sort(np.argsort(np.array(score))[::-1][:n])
        art_ext = [orig_art[ind] for ind in idx]
        art_ext = " ".join(art_ext)
        art_ext_list.append(art_ext)

    val_df_ext = pd.DataFrame()
    val_df_ext["extracted_article"] = art_ext_list
    val_df_ext.set_index(val_df.index, inplace=True)
    val_df_ext.to_csv('val/val_ext_'+str(k)+'.csv')

100%|██████████| 5/5 [13:42<00:00, 164.42s/it]


In [12]:
for k in tqdm(k_list):
    art_ext_list = []
    for i in tqdm(test_df.index, position=0):
        news = test_df.loc[i]
        art = news['article']
        high = news['highlights']
        art = sent_tokenize(art)
        orig_art = art

        art_form = []
        stop = stopwords.words('english')
        punc = string.punctuation

        for sent in art:
            sent_form = []
            words = word_tokenize(sent)
            for word in words:
                if word not in stop and word not in punc:
                    sent_form.append(word.lower())
            art_form.append(sent_form)

        art_form_1 = []
        art_form_len = []
        lemma = WordNetLemmatizer()

        for sent in art_form:
            words = []
            art_form_len.append(len(sent))
            for word in sent:
                lemma_word = lemma.lemmatize(word)
                words.append(lemma_word)
            art_form_1.append(words)

        art_form = art_form_1

        art_form_1 = []
        for sen in art_form:
            s = " ".join(sen)
            art_form_1.append(s)

        art_form = art_form_1

        vect = TfidfVectorizer()
        tf_idf_mat = vect.fit_transform(art_form)
        score = np.sum(tf_idf_mat, axis=1)
        score = np.ravel(score)
        n = max(int(len(art_form_len)/k),1)
        idx = np.sort(np.argsort(np.array(score))[::-1][:n])
        art_ext = [orig_art[ind] for ind in idx]
        art_ext = " ".join(art_ext)
        art_ext_list.append(art_ext)

    test_df_ext = pd.DataFrame()
    test_df_ext["extracted_article"] = art_ext_list
    test_df_ext.set_index(test_df.index, inplace=True)
    test_df_ext.to_csv('test/test_ext_'+str(k)+'.csv')

100%|██████████| 5/5 [11:57<00:00, 143.52s/it]
